In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Oct 22 00:37:10 2020

@author: charl
"""

#%%
import os
os.chdir('C:\\Users\\charl\\Desktop\\courses\\AN6001-AI & BIG DATA IN BUSINESS\\pro\\model')

import pandas as pd
import numpy as np
df = pd.read_csv('train.csv').fillna(' ')
df.describe()
import nltk
nltk.download('stopwords')

import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import stopwords


from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import punkt
nltk.download('punkt')
stop_words = stopwords.words('english')

#%%
# string
import string
special = string.punctuation
# remove URL
import re
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub('',text)

df['text']=df['text'].apply(lambda x : remove_URL(x))

## Removing HTML Tags

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

df['text']=df['text'].apply(lambda x: remove_html(x))

## Removing Punctuations

def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

df['text']=df['text'].apply(lambda x : remove_punct(x))

text = df['text']

#%%
list_corpus = df["text"].tolist()
list_labels = df["target"].tolist()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, test_size=0.2, 
                                                                                random_state=2020)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn import model_selection

#%% CounterVector---LG
from sklearn.feature_extraction.text import CountVectorizer
def cv(data):
    count_vectorizer = CountVectorizer()

    emb = count_vectorizer.fit_transform(data)

    return emb, count_vectorizer

X_train_counts, count_vectorizer = cv(X_train)
X_test_counts = count_vectorizer.transform(X_test)

model1 = LogisticRegression()
model1.fit(X_train_counts, y_train)

y_pred_train=model1.predict(X_train_counts)
y_pred = model1.predict(X_test_counts)
cm_train=confusion_matrix(y_train,y_pred_train)
acc1=(cm_train[0,0]+cm_train[1,1])/sum(sum(cm_train))
print(f'Accuracy of trainset is {acc1}')
cm=confusion_matrix(y_test, y_pred)
import seaborn as sns
import matplotlib.pyplot as plt
acc=round((cm[0,0]+cm[1,1])/sum(sum(cm))*100,2)
recall=round((cm[1,1]/(cm[1,0]+cm[1,1]))*100,2)

plt.figure(figsize=(4.5,3))
sns.heatmap(cm, annot=True, linewidth=0.5, fmt='.0f', cmap='viridis')
plt.title(f'LG CountVector \n Accuracy: {acc}% & Recall Rate: {recall}%')
plt.xlabel('Predict')
plt.ylabel('Actual')
plt.show()


from sklearn import model_selection
list_corpus_counts = count_vectorizer.transform(list_corpus)
cv_score = np.mean(model_selection.cross_val_score(model1, list_corpus_counts, df['target'], cv=3, scoring='f1'))
print(f'The Cross-validation accuracy is {round(cv_score*100,2)}%')


#%% TFIDF--LG
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf(data):
    tfidf_vectorizer = TfidfVectorizer()

    train = tfidf_vectorizer.fit_transform(data)

    return train, tfidf_vectorizer


X_train_tfidf, tfidf_vectorizer = tfidf(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

model1 = LogisticRegression()
model1.fit(X_train_tfidf, y_train)

y_pred_train=model1.predict(X_train_tfidf)
y_pred = model1.predict(X_test_tfidf)
cm_train=confusion_matrix(y_train,y_pred_train)
acc1=(cm_train[0,0]+cm_train[1,1])/sum(sum(cm_train))
print(f'Accuracy of trainset is {acc1}')
cm=confusion_matrix(y_test, y_pred)
import seaborn as sns
import matplotlib.pyplot as plt
acc=round((cm[0,0]+cm[1,1])/sum(sum(cm))*100,2)
recall=round((cm[1,1]/(cm[1,0]+cm[1,1]))*100,2)

plt.figure(figsize=(4.5,3))
sns.heatmap(cm, annot=True, linewidth=0.5, fmt='.0f', cmap='viridis')
plt.title(f'LG TFIDF \n Accuracy: {acc}% & Recall Rate: {recall}%')
plt.xlabel('Predict')
plt.ylabel('Actual')
plt.show()

from sklearn import model_selection
list_corpus_tfidf = tfidf_vectorizer.transform(list_corpus)
cv_score = np.mean(model_selection.cross_val_score(model1, list_corpus_tfidf, df['target'], cv=3, scoring='f1'))
print(f'The Cross-validation accuracy is {round(cv_score*100,2)}%')


#%% CountVector---NB
from sklearn.naive_bayes import MultinomialNB
from sklearn import model_selection
# Fitting a simple Naive Bayes on Counts
model2 = MultinomialNB()

model2.fit(X_train_counts, y_train)

y_pred_train=model2.predict(X_train_counts)
y_pred = model2.predict(X_test_counts)
cm_train=confusion_matrix(y_train,y_pred_train)
acc1=(cm_train[0,0]+cm_train[1,1])/sum(sum(cm_train))
print(f'Accuracy of trainset is {acc1}')
cm=confusion_matrix(y_test, y_pred)
acc=round((cm[0,0]+cm[1,1])/sum(sum(cm)),4)*100
recall=round((cm[1,1]/(cm[1,0]+cm[1,1]))*100,2)

plt.figure(figsize=(4.5,3))
sns.heatmap(cm, annot=True, linewidth=0.5, fmt='.0f', cmap='viridis')
plt.title(f'NB CountVector \n Accuracy: {acc}% & Recall Rate: {recall}%')
plt.xlabel('Predict')
plt.ylabel('Actual')
plt.show()

cv_score = np.mean(model_selection.cross_val_score(model2, list_corpus_counts, df['target'], cv=3, scoring='f1_micro'))
print(f'The Cross-validation accuracy is {round(cv_score*100,2)}%')

#%% TFIDF--- NB
from sklearn.naive_bayes import MultinomialNB
from sklearn import model_selection
# Fitting a simple Naive Bayes on Counts
model2 = MultinomialNB()

model2.fit(X_train_tfidf, y_train)

y_pred_train=model2.predict(X_train_tfidf)
y_pred = model2.predict(X_test_tfidf)
cm_train=confusion_matrix(y_train,y_pred_train)
acc1=(cm_train[0,0]+cm_train[1,1])/sum(sum(cm_train))
print(f'Accuracy of trainset is {acc1}')
cm=confusion_matrix(y_test, y_pred)
acc=round((cm[0,0]+cm[1,1])/sum(sum(cm))*100,2)
recall=round((cm[1,1]/(cm[1,0]+cm[1,1]))*100,2)

plt.figure(figsize=(4.5,3))
sns.heatmap(cm, annot=True, linewidth=0.5, fmt='.0f', cmap='viridis')
plt.title(f'NB TFIDF \n Accuracy: {acc}% & Recall Rate: {recall}%')
plt.xlabel('Predict')
plt.ylabel('Actual')
plt.show()

from sklearn import model_selection
cv_score = np.mean(model_selection.cross_val_score(model2, list_corpus_tfidf, df['target'], cv=3, scoring='f1'))
print(f'The Cross-validation accuracy is {round(cv_score*100,2)}%')